# Parameter optimization

In [6]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'Trend']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {
                            'use_vol': [-1], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'second_candle': [0.5], # np.arange(0.5, 1, 0.05),
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [48], # np.arange(2, 102, 2),
                            'low_bound': [0]
                            }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [01:26<00:00, 86.45s/it]


# Check local statistics

In [7]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_rank', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_Trend,-1,1,5,0.5,0.5,0.5,48,0,1024,1.553154,0.300833,566.429867,308.053333,1.1556,1.198,1.271,1.2969,1.3196,1.3808,1.518,1.5733,1.5805,1.6034,1.5933,1.6041,1.5987,1.595,1.5974,1.6499,1.7412,1.7671,1.775,1.7425,1.6906,1.6664,1.6688,1.6886,0.02,0.04,0.06,0.08,0.1,0.13,0.15,0.17,0.2,0.22,0.25,0.28,0.31,0.34,0.37,0.39,0.43,0.47,0.49,0.5,0.52,0.53,0.56,0.61


In [4]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_rank', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
100,Pattern_Trend,0,1,5,0.50,0.50,0.5,28,0,1026,1.469958,0.229167,482.177250,235.125000,1.1058,1.1274,1.1978,1.2207,1.2432,1.3010,1.3941,1.4415,1.4496,1.4666,1.4703,1.4913,1.4991,1.5109,1.5223,1.5741,1.6841,1.7058,1.7029,1.6782,1.6271,1.6093,1.6117,1.6442,0.02,0.04,0.05,0.07,0.08,0.10,0.12,0.13,0.14,0.16,0.17,0.20,0.23,0.24,0.27,0.29,0.33,0.37,0.39,0.39,0.42,0.40,0.42,0.47
0,Pattern_Trend,-1,1,5,0.50,0.50,0.5,28,0,880,1.530042,0.229583,466.436667,202.033333,1.1359,1.1742,1.2513,1.2686,1.2912,1.3481,1.4627,1.5184,1.5241,1.5420,1.5353,1.5565,1.5459,1.5547,1.5636,1.6265,1.7515,1.7775,1.7752,1.7491,1.6991,1.6800,1.6824,1.7072,0.02,0.03,0.04,0.06,0.08,0.09,0.11,0.12,0.14,0.16,0.18,0.20,0.23,0.24,0.27,0.29,0.33,0.37,0.39,0.39,0.41,0.42,0.45,0.49
110,Pattern_Trend,0,1,5,0.55,0.50,0.5,28,0,960,1.472479,0.246667,453.580000,236.800000,1.0758,1.1235,1.2002,1.2298,1.2525,1.3562,1.3924,1.4512,1.4522,1.4647,1.4744,1.4943,1.5024,1.5141,1.5290,1.5730,1.6915,1.7112,1.7124,1.6806,1.6219,1.6026,1.6038,1.6298,0.02,0.04,0.06,0.07,0.09,0.11,0.13,0.15,0.16,0.18,0.20,0.22,0.24,0.27,0.30,0.32,0.36,0.39,0.42,0.41,0.44,0.42,0.44,0.48
101,Pattern_Trend,0,1,5,0.50,0.55,0.5,28,0,954,1.466225,0.202083,444.778650,192.787500,1.0959,1.1305,1.1946,1.2239,1.2448,1.3064,1.4097,1.4358,1.4427,1.4535,1.4472,1.4789,1.4809,1.4941,1.5094,1.5670,1.6825,1.7032,1.7065,1.6802,1.6325,1.6149,1.6104,1.6439,0.02,0.03,0.04,0.05,0.06,0.08,0.09,0.10,0.11,0.12,0.14,0.16,0.19,0.20,0.23,0.26,0.30,0.34,0.36,0.37,0.39,0.38,0.39,0.44
10,Pattern_Trend,-1,1,5,0.55,0.50,0.5,28,0,823,1.537504,0.241667,442.365929,198.891667,1.1225,1.1822,1.2656,1.2887,1.3091,1.4184,1.4678,1.5265,1.5226,1.5386,1.5397,1.5617,1.5520,1.5594,1.5721,1.6266,1.7623,1.7875,1.7902,1.7564,1.6975,1.6755,1.6781,1.6991,0.02,0.03,0.05,0.07,0.09,0.11,0.12,0.14,0.15,0.17,0.19,0.22,0.24,0.26,0.29,0.31,0.34,0.38,0.40,0.40,0.42,0.44,0.46,0.50
1,Pattern_Trend,-1,1,5,0.50,0.55,0.5,28,0,824,1.535517,0.196250,441.265733,161.710000,1.1338,1.1875,1.2562,1.2802,1.3018,1.3630,1.4926,1.5260,1.5337,1.5454,1.5292,1.5591,1.5406,1.5489,1.5604,1.6298,1.7612,1.7843,1.7851,1.7544,1.7066,1.6877,1.6820,1.7029,0.01,0.02,0.03,0.04,0.05,0.07,0.08,0.09,0.10,0.12,0.13,0.16,0.18,0.20,0.22,0.25,0.29,0.33,0.35,0.35,0.38,0.39,0.41,0.46
120,Pattern_Trend,0,1,5,0.60,0.50,0.5,28,0,892,1.474750,0.267083,423.477000,238.238333,1.0667,1.1093,1.1824,1.2139,1.2440,1.3618,1.3994,1.4583,1.4575,1.4708,1.4874,1.5074,1.4984,1.5115,1.5314,1.5777,1.6943,1.7241,1.7218,1.6869,1.6218,1.6107,1.6142,1.6423,0.03,0.04,0.06,0.08,0.10,0.12,0.15,0.16,0.18,0.20,0.22,0.25,0.27,0.29,0.32,0.34,0.38,0.42,0.45,0.44,0.47,0.45,0.47,0.52
20,Pattern_Trend,-1,1,5,0.60,0.50,0.5,28,0,769,1.547800,0.256667,421.258200,197.376667,1.1230,1.1658,1.2442,1.2717,1.3045,1.4264,1.4712,1.5286,1.5212,1.5423,1.5565,1.5791,1.5652,1.5739,1.5912,1.6478,1.7794,1.8169,1.8154,1.7780,1.7121,1.7000,1.7052,1.7276,0.02,0.03,0.05,0.07,0.09,0.11,0.13,0.14,0.16,0.18,0.20,0.23,0.25,0.28,0.30,0.33,0.36,0.40,0.43,0.43,0.45,0.47,0.50,0.55
111,Pattern_Trend,0,1,5,0.55,0.55,0.5,28,0,876,1.474150,0.226667,415.355400,198.560000,1.0649,

In [13]:
# LR timeperiod
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
44,Pattern_Trend,-1,1,5,0.5,0.5,0.5,90,0,1097,1.607892,0.341250,666.857158,374.351250,1.1649,1.2274,1.3200,1.3367,1.3543,1.4085,1.5456,1.6118,1.6339,1.6576,1.6617,1.6730,1.6730,1.6678,1.6755,1.7271,1.8147,1.8317,1.8439,1.8053,1.7580,1.7351,1.7216,1.7403,0.02,0.05,0.06,0.09,0.11,0.14,0.16,0.19,0.22,0.25,0.28,0.32,0.35,0.38,0.41,0.44,0.48,0.53,0.56,0.58,0.60,0.62,0.65,0.70
42,Pattern_Trend,-1,1,5,0.5,0.5,0.5,86,0,1090,1.621317,0.340417,677.235167,371.054167,1.1712,1.2345,1.3242,1.3390,1.3616,1.4182,1.5557,1.6178,1.6428,1.6707,1.6787,1.6918,1.6912,1.6854,1.6918,1.7414,1.8284,1.8461,1.8614,1.8244,1.7778,1.7543,1.7402,1.7630,0.02,0.04,0.06,0.08,0.11,0.13,0.16,0.18,0.22,0.25,0.28,0.31,0.34,0.37,0.41,0.44,0.48,0.53,0.57,0.58,0.61,0.63,0.66,0.71
46,Pattern_Trend,-1,1,5,0.5,0.5,0.5,94,0,1109,1.591396,0.340000,655.857979,377.060000,1.1553,1.2077,1.2978,1.3164,1.3367,1.3920,1.5292,1.5965,1.6208,1.6415,1.6448,1.6564,1.6559,1.6514,1.6597,1.7110,1.7977,1.8131,1.8256,1.7875,1.7417,1.7200,1.7079,1.7269,0.02,0.05,0.06,0.09,0.11,0.14,0.16,0.19,0.22,0.25,0.28,0.32,0.35,0.37,0.41,0.44,0.48,0.53,0.56,0.57,0.60,0.62,0.65,0.69
45,Pattern_Trend,-1,1,5,0.5,0.5,0.5,92,0,1104,1.596879,0.340000,658.954600,375.360000,1.1577,1.2156,1.3063,1.3249,1.3447,1.3993,1.5359,1.6019,1.6247,1.6459,1.6495,1.6610,1.6603,1.6556,1.6638,1.7154,1.8024,1.8193,1.8317,1.7937,1.7474,1.7250,1.7121,1.7310,0.02,0.05,0.06,0.09,0.11,0.14,0.16,0.19,0.22,0.25,0.28,0.32,0.35,0.37,0.41,0.44,0.48,0.53,0.56,0.57,0.60,0.62,0.65,0.69
43,Pattern_Trend,-1,1,5,0.5,0.5,0.5,88,0,1092,1.620463,0.335833,677.545050,366.730000,1.1720,1.2353,1.3265,1.3402,1.3602,1.4132,1.5507,1.6168,1.6417,1.6696,1.6777,1.6907,1.6901,1.6844,1.6913,1.7409,1.8280,1.8458,1.8601,1.8234,1.7770,1.7536,1.7396,1.7623,0.02,0.04,0.06,0.08,0.11,0.13,0.16,0.18,0.21,0.24,0.27,0.31,0.34,0.37,0.40,0.43,0.48,0.53,0.56,0.57,0.60,0.62,0.65,0.70
37,Pattern_Trend,-1,1,5,0.5,0.5,0.5,76,0,1064,1.631071,0.330417,671.459367,351.563333,1.1678,1.2436,1.3346,1.3546,1.3757,1.4338,1.5745,1.6338,1.6513,1.6789,1.6835,1.6975,1.6979,1.6931,1.6973,1.7494,1.8421,1.8602,1.8716,1.8362,1.7854,1.7602,1.7492,1.7735,0.02,0.04,0.06,0.08,0.10,0.13,0.16,0.18,0.21,0.24,0.27,0.30,0.33,0.36,0.39,0.42,0.47,0.52,0.55,0.56,0.59,0.61,0.65,0.69
48,Pattern_Trend,-1,1,5,0.5,0.5,0.5,98,0,1117,1.587162,0.329583,655.860512,368.144583,1.1658,1.2110,1.3000,1.3192,1.3379,1.3919,1.5281,1.5955,1.6189,1.6363,1.6388,1.6505,1.6507,1.6475,1.6558,1.7069,1.7934,1.8077,1.8165,1.7776,1.7287,1.7056,1.6943,1.7133,0.02,0.04,0.06,0.08,0.11,0.13,0.16,0.18,0.22,0.24,0.27,0.31,0.34,0.36,0.40,0.42,0.47,0.51,0.55,0.56,0.58,0.60,0.63,0.67
47,Pattern_Trend,-1,1,5,0.5,0.5,0.5,96,0,1115,1.585888,0.329583,653.264563,367.485417,1.1622,1.2082,1.2978,1.3169,1.3359,1.3900,1.5264,1.5940,1.6170,1.6344,1.6373,1.6490,1.6493,1.6461,1.6543,1.7054,1.7915,1.8058,1.8145,1.7756,1.7307,1.7076,1.6962,1.7152,0.02,0.04,0.06,0.08,0.11,0.13,0.16,0.18,0.22,0.24,0.27,0.31,0.34,0.36,0.40,0.42,0.47,0.51,0.55,0.56,0.58,0.60,0.63,0.67
49,Pattern_Trend,-1,1,5,0.5,0.5,0.5,100,0,1123,1.582042,0.329167,653.632792,369.654167,1.1730,1.2167,

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,Pattern_Trend,1,1,5,0.8,0.5,0.5,28,0,117,2.721938,1.287083,201.466688,150.588750,1.6343,1.8121,1.8077,2.1569,2.2423,2.2469,2.2988,2.3666,2.3793,2.4783,2.5767,2.7266,2.7558,2.8295,2.9710,3.0502,3.0906,3.2132,3.2524,3.3021,3.2832,3.3928,3.6759,3.7833,0.07,0.15,0.23,0.32,0.41,0.51,0.61,0.70,0.78,0.90,1.04,1.16,1.29,1.42,1.56,1.71,1.86,1.99,2.14,2.27,2.41,2.29,2.46,2.61
0,Pattern_Trend,-1,1,5,0.8,0.5,0.5,28,0,973,1.991688,1.038333,964.911938,1010.298333,1.3548,1.3729,1.3539,1.4297,1.4860,1.5418,1.6474,1.7221,1.7658,1.7952,1.8123,1.8344,2.1827,2.2239,2.2573,2.3209,2.3618,2.4302,2.4403,2.4636,2.4844,2.4983,2.4990,2.5218,0.08,0.16,0.24,0.33,0.41,0.50,0.58,0.67,0.75,0.83,0.92,1.00,1.07,1.16,1.25,1.34,1.43,1.52,1.59,1.67,1.74,1.82,1.89,1.97
1,Pattern_Trend,0,1,5,0.8,0.5,0.5,28,0,1267,2.022196,1.019583,1295.122121,1291.812083,1.3566,1.3828,1.3841,1.4915,1.5454,1.5976,1.6958,1.7785,1.8123,1.8405,1.8598,1.8944,2.1758,2.2282,2.2807,2.3418,2.3830,2.4511,2.4514,2.4734,2.4922,2.5141,2.5347,2.5670,0.08,0.15,0.23,0.31,0.39,0.47,0.56,0.64,0.72,0.80,0.88,0.96,1.04,1.13,1.22,1.32,1.42,1.51,1.59,1.66,1.74,1.80,1.88,1.97


In [3]:
# high/low bound
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
9,Pattern_Trend,-1,3,5,0.5,0.5,0.5,90,0,805,1.516488,0.423750,415.772438,341.118750,1.1238,1.0586,1.1892,1.2501,1.2691,1.2827,1.4091,1.4491,1.5176,1.5436,1.5877,1.5787,1.5452,1.5596,1.5832,1.6570,1.7399,1.7647,1.7597,1.7191,1.7068,1.6922,1.6927,1.7163,0.04,0.08,0.11,0.15,0.18,0.21,0.25,0.28,0.31,0.35,0.38,0.41,0.45,0.47,0.50,0.53,0.58,0.62,0.65,0.66,0.69,0.72,0.74,0.81
8,Pattern_Trend,-1,3,4,0.5,0.5,0.5,90,0,618,1.404229,0.397083,249.813625,245.397500,1.0842,0.9740,1.0469,1.1165,1.1634,1.1895,1.3276,1.3671,1.4413,1.4629,1.5249,1.5167,1.4726,1.4923,1.5155,1.5350,1.5627,1.5955,1.5955,1.5537,1.5374,1.5233,1.5359,1.5671,0.05,0.08,0.11,0.15,0.19,0.22,0.25,0.27,0.31,0.34,0.37,0.40,0.43,0.45,0.47,0.50,0.54,0.57,0.60,0.59,0.62,0.63,0.66,0.73
11,Pattern_Trend,-1,3,7,0.5,0.5,0.5,90,0,1156,1.580737,0.353333,671.332550,408.453333,1.1421,1.1353,1.2486,1.2997,1.3252,1.3825,1.5058,1.5363,1.5912,1.6143,1.6399,1.6339,1.6081,1.6207,1.6393,1.6994,1.7698,1.8360,1.8183,1.7920,1.7757,1.7696,1.7675,1.7865,0.03,0.06,0.09,0.12,0.15,0.18,0.21,0.23,0.27,0.29,0.31,0.34,0.36,0.38,0.40,0.43,0.47,0.51,0.53,0.55,0.59,0.62,0.65,0.71
10,Pattern_Trend,-1,3,6,0.5,0.5,0.5,90,0,977,1.540050,0.347917,527.628850,339.914583,1.1314,1.0826,1.1972,1.2400,1.2656,1.3077,1.4535,1.4774,1.5471,1.5843,1.6180,1.6049,1.5767,1.5862,1.6040,1.6704,1.7495,1.7908,1.7707,1.7433,1.7369,1.7354,1.7334,1.7542,0.04,0.07,0.10,0.13,0.16,0.18,0.21,0.23,0.26,0.29,0.31,0.34,0.36,0.38,0.40,0.43,0.47,0.50,0.52,0.53,0.56,0.59,0.61,0.68
1,Pattern_Trend,-1,1,5,0.5,0.5,0.5,90,0,1173,1.604625,0.337500,709.225125,395.887500,1.1742,1.2275,1.3084,1.3301,1.3525,1.4060,1.5432,1.6104,1.6300,1.6509,1.6574,1.6641,1.6643,1.6641,1.6713,1.7244,1.8083,1.8260,1.8388,1.8026,1.7588,1.7303,1.7238,1.7436,0.02,0.04,0.06,0.09,0.11,0.13,0.16,0.19,0.22,0.25,0.28,0.32,0.35,0.38,0.41,0.43,0.48,0.52,0.55,0.56,0.59,0.61,0.65,0.70
5,Pattern_Trend,-1,2,5,0.5,0.5,0.5,90,0,1136,1.587421,0.336667,667.310067,382.453333,1.1762,1.2342,1.3145,1.3408,1.3650,1.4141,1.5421,1.5979,1.6222,1.6351,1.6339,1.6353,1.6279,1.6271,1.6351,1.6965,1.7718,1.7916,1.8056,1.7748,1.7313,1.7052,1.6995,1.7204,0.03,0.05,0.07,0.10,0.12,0.15,0.18,0.20,0.24,0.27,0.29,0.33,0.35,0.37,0.40,0.43,0.47,0.51,0.54,0.54,0.57,0.58,0.62,0.67
0,Pattern_Trend,-1,1,4,0.5,0.5,0.5,90,0,985,1.537025,0.297083,528.969625,292.627083,1.1570,1.1986,1.2322,1.2547,1.2913,1.3569,1.5015,1.5710,1.5864,1.6040,1.6179,1.6277,1.6267,1.6284,1.6328,1.6435,1.6908,1.7133,1.7322,1.6957,1.6465,1.6178,1.6187,1.6430,0.02,0.04,0.05,0.07,0.10,0.12,0.14,0.17,0.20,0.22,0.25,0.29,0.31,0.33,0.36,0.39,0.42,0.46,0.49,0.49,0.51,0.52,0.57,0.61
4,Pattern_Trend,-1,2,4,0.5,0.5,0.5,90,0,950,1.518987,0.295833,493.038125,281.041667,1.1592,1.2056,1.2370,1.2645,1.3025,1.3656,1.5016,1.5602,1.5807,1.5885,1.5934,1.5970,1.5870,1.5879,1.5930,1.6165,1.6523,1.6773,1.6974,1.6667,1.6177,1.5916,1.5934,1.6191,0.03,0.05,0.06,0.09,0.11,0.14,0.16,0.18,0.21,0.24,0.26,0.29,0.31,0.33,0.36,0.38,0.41,0.45,0.47,0.47,0.49,0.50,0.54,0.57
3,Pattern_Trend,-1,1,7,0.5,0.5,0.5,90,0,1522,1.627200,0.295417,954.598400,449.624167,1.1681,1.2426,1.3224,1.3451

# Research parameter influence

In [3]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.546973,2.873992,1061.440504,5577.185392,291081
1,0,1.387825,2.958303,865.087474,6598.829933,334592
2,1,1.387787,2.958333,865.153912,6600.041667,334650


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [4]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'Trend']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                 'Pattern': {
                            'use_vol': [0], 
                            'window_low_bound': [1], 
                            'window_high_bound': [5],
                            'first_candle': [0.5],
                            'second_candle': [0.5],
                            'third_candle': [0.5]
                            },
                'Trend': {
                            'timeperiod': [90], 
                            'low_bound': [0]
                            }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)